# Building analysis with ETABS

## Connect to ETABS

In [2]:
import sys
import os
# Get the current working directory
current_folder = os.getcwd()
# Go up two directories
parent_folder = os.path.dirname(os.path.dirname(current_folder))
# Append the parent folder to the system path (for .ipynb files)
sys.path.append(parent_folder)
from etabsninja.interface import connect_to_etabs, test_etabs_connection
# Run this one time and that's it
SapModel,EtabsObject = connect_to_etabs()
test_etabs_connection()

ETABS connection test successful


## Import Database

In [5]:
from etabsninja.get_database import DatabaseTables

## Mallado y diafragmas

In [6]:
table_name = "Area Assignments - Wall Auto Mesh Options"
Database =DatabaseTables(SapModel)
df=Database.read(table_name,to_dataframe=True)
assert df is not None, "DataFrame should not be None"
unique_mesh_options = df['MeshOption'].unique()
print('Cantidad de wall mesh options:', unique_mesh_options)

Cantidad de wall mesh options: ['Auto Rectangular']


In [7]:
table_name = "Area Assignments - Floor Auto Mesh Options"
Database =DatabaseTables(SapModel)
df=Database.read(table_name,to_dataframe=True)
assert df is not None, "DataFrame should not be None"
unique_mesh_options = df['MeshOption'].unique()
print('Cantidad de floor mesh options:', unique_mesh_options)

Cantidad de floor mesh options: ['Default']


In [ ]:
table_name = "Area Assignments - Diaphragms"
Database =DatabaseTables(SapModel)
df=Database.read(table_name,to_dataframe=True)
assert df is not None, "DataFrame should not be None"
summary = df.groupby('Diaphragm').agg({'Story': lambda x: list(x), 'Diaphragm': 'size'}).rename(columns={'Diaphragm': 'Count'})
print('Cantidad de diafragmas:\n', summary)
summary.to_excel('T1-Diafragmas.xlsx', index=True)  # Set index=False to exclude the index from the Excel file

## Stiffness modifiers

In [ ]:
table_name = "Area Assignments - Stiffness Modifiers"
Database =DatabaseTables(SapModel)
df=Database.read(table_name,to_dataframe=True)
assert df is not None, "DataFrame should not be None"
# Filter rows where 'Label' starts with 'F' and 'W'
filtered_df_F = df[df['Label'].str.startswith('F')]
filtered_df_W = df[df['Label'].str.startswith('W')]
# Group by 'Story' and get unique values of 'f11Mod'
summary = filtered_df_F.groupby('Story')[['f11Mod', 'f22Mod', 'f12Mod', 'm11Mod', 'm22Mod', 'm12Mod']].agg(pd.Series.unique)
print('Modificadores de rigidez en elementos de losa por piso:\n', summary)
# Get unique values for the entire DataFrame
summary_total = filtered_df_F[['f11Mod', 'f22Mod', 'f12Mod', 'm11Mod', 'm22Mod', 'm12Mod']].agg(pd.Series.unique)
print('Modificadores de rigidez en elementos de losa en el edificio:\n',summary_total)
summary = filtered_df_W.groupby('Story')[['f11Mod', 'f22Mod', 'f12Mod', 'm11Mod', 'm22Mod', 'm12Mod']].agg(pd.Series.unique)
print('Modificadores de rigidez en elementos de muro por piso:\n', summary)
summary_total = filtered_df_W[['f11Mod', 'f22Mod', 'f12Mod', 'm11Mod', 'm22Mod', 'm12Mod']].agg(pd.Series.unique)
print('Modificadores de rigidez en elementos de muro en el edificio:\n',summary_total)
summary.to_excel('T1-Muros.xlsx', index=True) 


In [ ]:
table_name = "Frame Assignments - Property Modifiers"
Database =DatabaseTables(SapModel)
df=Database.read(table_name,to_dataframe=True)
assert df is not None, "DataFrame should not be None"
# Filter rows where 'Label' starts with 'B' and 'C'
filtered_df_B = df[df['Label'].str.startswith('B')]
filtered_df_C = df[df['Label'].str.startswith('C')]
# Group by 'Story' and get unique values of 'f11Mod'
summary = filtered_df_B.groupby('Story')[['AMod', 'A2Mod', 'A3Mod', 'JMod', 'I2Mod', 'I3Mod','MMod']].agg(pd.Series.unique)
print('Modificadores de rigidez en elementos de viga por piso:\n', summary)
# Get unique values for the entire DataFrame
summary_total = filtered_df_B[['AMod', 'A2Mod', 'A3Mod', 'JMod', 'I2Mod', 'I3Mod','MMod']].agg(pd.Series.unique)
summary = filtered_df_C.groupby('Story')[['AMod', 'A2Mod', 'A3Mod', 'JMod', 'I2Mod', 'I3Mod','MMod']].agg(pd.Series.unique)
summary_total = filtered_df_C[['AMod', 'A2Mod', 'A3Mod', 'JMod', 'I2Mod', 'I3Mod','MMod']].agg(pd.Series.unique)
print('Modificadores de rigidez en elementos de viga en el edificio:\n',summary_total)
print('Modificadores de rigidez en elementos de columna por piso:\n', summary)
print('Modificadores de rigidez en elementos de columna en el edificio:\n',summary_total)